#### Configure Chinook DB from Google
- url : https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db

In [ ]:
import requests
url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
response = requests.get(url)
if response.status_code == 200 :
    # Open local file in write mode (binary)
    with open("Chinook.db", "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved as Chinook.db")
else:
    print(f"Error connecting to DB : {response.status_code}")

In [ ]:
# Define SQL Database wrapper available in langchain
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(f"db.dialect : {db.dialect}")
print(f"db.get_usable_tables_names : {db.get_usable_table_names()}")
db.run("SELECT * FROM Artist LIMIT 10;")

In [ ]:
# Utility Functions, Error handling
